In [36]:
import postal
import sys
import os
from datetime import datetime, timezone
from pathlib import Path
import pandas as pd
import re
from postal.expand import expand_address
from postal.parser import parse_address
from dotenv import load_dotenv
load_dotenv('/home/james/PDS/client_data_feeds/realestate/airflow/.env')

# Add airflow directory and parent directory to path (for pyRealtor)
sys.path.insert(0, '/home/james/PDS/client_data_feeds/realestate/airflow')
sys.path.insert(0, '/home/james/PDS/client_data_feeds/realestate')
from include.db.connections import get_master_db_connection

conn = get_master_db_connection()

In [37]:
sql = """SELECT * FROM properties WHERE postal_code = 'AAA AAA';"""
df = pd.read_sql(sql, conn)
print(df)

Empty DataFrame
Columns: [id, address_id, address_number, country, lat, lon, postal_code, street_name, municipality, province_state, geom, updated_at]
Index: []


/tmp/ipykernel_1186593/2699684636.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql, conn)


In [5]:
df.columns

Index(['id', 'address_id', 'address_number', 'country', 'lat', 'lon',
       'postal_code', 'street_name', 'municipality', 'province_state', 'geom',
       'updated_at'],
      dtype='object')

In [ ]:
import requests
import re
GOOGLE_GEOCODE_API_KEY=''
api_key = GOOGLE_GEOCODE_API_KEY

In [6]:
df['formatted_address'] = None

In [38]:
len(df)

0

In [30]:
df['postal_code'].value_counts()

postal_code
AAA AAA    4753
Name: count, dtype: int64

In [16]:
test_df = df.copy()
test_df = test_df.head(5)

In [18]:
test_df.columns

Index(['id', 'address_id', 'address_number', 'country', 'lat', 'lon',
       'postal_code', 'street_name', 'municipality', 'province_state', 'geom',
       'updated_at', 'formatted_address'],
      dtype='object')

In [27]:
def get_real_postal_code(row, timeout_s=5):
    address = row['address_number'] + ' ' + row['street_name'] + ', ' + row['municipality'] + ', ' + row['province_state'] + ', ' + row['country']
    url = (
        "https://maps.googleapis.com/maps/api/geocode/json"
        f"?address={requests.utils.quote(address)}"
        f"&key={api_key}"
    )
    resp = requests.get(url, timeout=timeout_s)
    resp.raise_for_status()
    payload = resp.json()
    if not payload["results"]:
        # No results found, keep original postal code
        return row
    
    # Try each result until we find one with a postal code
    for result in payload["results"]:
        formatted_address = result["formatted_address"]
        row['formatted_address'] = formatted_address
        
        # Extract postal code from address_components
        postal_code = None
        for component in result["address_components"]:
            if "postal_code" in component["types"]:
                postal_code = component["long_name"]
                break
        
        if postal_code:
            row['postal_code'] = postal_code
            return row
    
    # If no postal code found in any result, keep original
    return row

test_df = test_df.apply(get_real_postal_code, axis=1)


In [31]:
df = df.apply(get_real_postal_code, axis=1)

In [32]:
df.head()

,id,address_id,address_number,country,lat,lon,postal_code,street_name,municipality,province_state,geom,updated_at,formatted_address
0,21f23474-f9d8-4074-a065-f643e32d14ce,747657,20291,Canada,43.930269,-80.009970,L7K 2C3,HORSESHOE HILL ROAD,Caledon,Ontario,0101000020E6100000AE92E55AA30054C061C7270E13F7...,2026-01-19 18:35:22.398386+00:00,"20291 Horseshoe Hill Rd, Caledon, ON L7K 2C3, ..."
1,aba2a0de-9063-4ba4-b713-3fea929d60b8,753409,19508,Canada,43.920701,-79.982577,L7K 2E5,ST. ANDREW'S ROAD,Caledon,Ontario,0101000020E61000004159A188E2FE53C08E035689D9F5...,2026-01-19 18:35:22.398386+00:00,"19508 St Andrews Rd, Caledon East, ON L7K 2E5,..."
2,56eaae2e-11eb-47b7-b2ea-e83625ec6610,753905,16275,Canada,43.904452,-79.842558,L7E 0X4,THE GORE ROAD,Caledon,Ontario,0101000020E6100000CE60FB79ECF553C025CE8F16C5F3...,2026-01-19 18:35:22.398386+00:00,"16275 The Gore Rd, Caledon East, ON L7E 0X4, C..."
3,4d801186-33a7-4959-b698-85cde4f83e5a,754829,9294,Canada,43.971471,-79.854837,L7E 0H4,FINNERTY SIDEROAD,Caledon,Ontario,0101000020E6100000F7995DA6B5F653C016DCF62A59FC...,2026-01-19 18:35:22.398386+00:00,"9294 Finnerty Side Rd, Caledon, ON L7E 0H4, Ca..."
4,b537f24e-448c-4bee-8fa1-ba5296eba13e,755046,16776,Canada,43.986994,-79.774617,AAA AAA,CALEDON/KING TOWN LINE N,Caledon,Ontario,0101000020E6100000E6F9A75293F153C0F9E1E8D455FE...,2026-01-19 18:35:22.398386+00:00,"Town Line, Caledon, ON, Canada"


In [33]:
df['postal_code'].value_counts()

postal_code
AAA AAA    601
N0B 1M0     63
V3Z 9V2     14
V4N 4P5     13
V3W 3M8     13
          ... 
V2X 4G6      1
V2X 5V1      1
V2X 9Z8      1
L7E 0H6      1
L7E 0H5      1
Name: count, Length: 2588, dtype: int64

In [34]:
#now we will update the DB with the new postal codes, if a postal code is still AAA AAA set to Unknown

update_sql = """
UPDATE properties
SET postal_code = %s
WHERE id = %s;
"""
conn = get_master_db_connection()
for index, row in df.iterrows():
    postal_code = row['postal_code']
    if postal_code == 'AAA AAA':
        postal_code = 'Unknown'
    property_id = row['id']
    with conn.cursor() as cursor:
        cursor.execute(update_sql, (postal_code, property_id))
    conn.commit()
conn.close()

